<div style="text-align: center; font-family: 'charter bt pro roman'; color: rgb(0, 65, 75);">
    <h1>
    GDP Revisions Datasets
    </h1>
</div>

<div style="text-align: center; font-family: 'charter bt pro roman'; color: rgb(0, 65, 75);">
    <h3>
        Documentation
        <br>
        ____________________
            </br>
    </h3>
</div>

<div style="text-align: center; font-family: 'PT Serif Pro Book'; color: rgb(0, 65, 75); font-size: 16px;">
    Jason Cruz
    <br>
    <a href="mailto:jj.cruza@up.edu.pe" style="color: rgb(0, 153, 123); font-size: 16px;">
        jj.cruza@up.edu.pe
    </a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
This <span style="color: rgb(0, 65, 75);">jupyter notebook</span> documents step-by-step the <b>construction of datasets</b> for the project <b>'Revisions and biases in preliminary GDP estimates in Peru'</b>.

This jupyter notebook goes from downloading the Weekly Notes (NS) from the Central Reserve Bank of Peru (BCRP), stored on their website as PDF files, to generating datasets of growth rates and revisions to Peru's GDP, loaded as tables to SQL. The NS contain the information on annual, quarterly and monthly GDP growth rates by economic sectors of Peru, while the main datasets that will be used for the data analysis of this project are generated in this jupyter notebook using big data and machine learning techniques.
</div>

<div style="font-family: Amaya; text-align: left; color: rgb(0, 65, 75); font-size:16px">The following <b>outline is functional</b>. By utilising the provided buttons, users are able to enhance their experience by browsing this script.<div/>

<div id="outilne">
   <!-- Contenido de la celda de destino -->
</div>

<div style="background-color: #272727; padding: 10px;">
<h2 style="text-align: left; font-family: 'charter'; color: #E0E0E0;">
    Outline
    </h2>
    <br>
    <a href="#libraries" style="color: #687EFF; font-size: 18px;">
        Libraries</a>
    <br>
    <a href="#setup" style="color: #687EFF; font-size: 18px;">
        Initial set-up</a>
    <br>
    <a href="#1" style="color: #687EFF; font-size: 18px;">
        1. PDF Downloader</a>
    <br>
    <a href="#2" style="color: #687EFF; font-size: 18px;">
        2. Data cleaning</a>
    <br>
    <a href="#2-1" style="color: rgb(0, 153, 123); font-size: 12px;">
        2.1. 'pdfplumber' demo.</a>
    <br>
    <a href="#2-1-1" style="color: #E0E0E0; font-size: 12px;">
        2.1.1. What data would we get if we used the default settings?.</a>   
    <br>
    <a href="#2-1-2" style="color: #E0E0E0; font-size: 12px;">
        2.1.2. Using custom '.extract_table' settings.</a>
    <br> 
    <a href="#2-2" style="color: rgb(0, 153, 123); font-size: 12px;">
        2.2. Extracting tables and generating dataframes (includes data cleanup).</a>
    <br>
    <a href="#3" style="color: #687EFF; font-size: 18px;">3. SQL Tables</a>
    <br>
    <a href="#3-1" style="color: rgb(0, 153, 123); font-size: 12px;">
        3.1. Annual Concatenation.</a>
    <br>
    <a href="#3-2" style="color: rgb(0, 153, 123); font-size: 12px;">
        3.2. Quarterly Concatenation.</a>
    <br>
    <a href="#3-3" style="color: rgb(0, 153, 123); font-size: 12px;">
        3.3. Monthly Concatenation.</a>
    <br>
    <a href="#3-4" style="color: rgb(0, 153, 123); font-size: 12px;">
        3.4. Loading SQL.</a>
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Any questions or issues regarding the coding, please <a href="mailto:jj.cruza@alum.up.edu.pe" style="color: rgb(0, 153, 123)">email Jason Cruz
    </a>.
    <div/>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px"">
    If you don't have the libraries below, please use the following code (as example) to install the required libraries.
    <div/>

In [ ]:
#!pip install os # Comment this code with "#" if you have already installed this library.

<div id="libraries">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark;">
    <h2>
    Libraries
    </h2>
    <div/>

In [1]:
# 1. PDF downloader

import os  # for file and directory manipulation
import random  # to generate random numbers
from selenium import webdriver  # for automating web browsers
from selenium.webdriver.common.by import By  # to locate elements on a webpage
from selenium.webdriver.support.ui import WebDriverWait  # to wait until certain conditions are met on a webpage.
from selenium.webdriver.support import expected_conditions as EC  # to define expected conditions
from selenium.common.exceptions import StaleElementReferenceException  # To handle exceptions related to elements on the webpage that are no longer available.
import pygame # Allows you to handle graphics, sounds and input events.

# 2. Extracting tables (and data cleaning)

import pdfplumber  # for extracting text and metadata from PDF files
import pandas as pd  # for data manipulation and analysis
import os  # for interacting with the operating system
import unicodedata  # for manipulating Unicode data
import re  # for regular expressions operations
from datetime import datetime  # for working with dates and times
import locale  # for locale-specific formatting of numbers, dates, and currencies


# 3. SQL tables

import psycopg2  # for interacting with PostgreSQL databases
from sqlalchemy import create_engine, text  # for creating and executing SQL queries using SQLAlchemy


pygame 2.5.2 (SDL 2.28.3, Python 3.12.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px">
    Back to the
    <a href="#outilne" style="color: #3d30a2;">
    outline.
    </a>
    <div/>

<div id="setup">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark;">
    <h2>
    Initial set-up
    </h2>
    <div/>

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px"> The following code lines will create folders in your current path, call them to import and export your outputs. <div/>

In [2]:
# Folder path to save downloaded PDF files

raw_pdf = 'raw_pdf' # to save raw data (.pdf).
if not os.path.exists(raw_pdf):
    os.mkdir(raw_pdf) # to create the folder (if it doesn't exist)

In [3]:
# Folder path to save text file with the names of already downloaded files

download_record = 'download_record'
if not os.path.exists(download_record):
    os.mkdir(download_record) # to create the folder (if it doesn't exist)

In [4]:
# Folder path to download the trimmed PDF files (these are PDF inputs for the extraction and cleanup code)

input_pdf = 'input_pdf'
if not os.path.exists(input_pdf):
    os.makedirs(input_pdf) # to create the folder (if it doesn't exist)

In [5]:
# Folder path to save PDF files containing only the pages of interest (where the GDP growth rate tables are located)

input_pdf_record = 'input_pdf_record'
if not os.path.exists(input_pdf_record):
    os.makedirs(input_pdf_record)

In [6]:
# Folder path to save dataframes generated record by year

dataframes_record = 'dataframes_record'
if not os.path.exists(dataframes_record):
    os.makedirs(dataframes_record) # to create the folder (if it doesn't exist)

In [7]:
# Folder path to save sound files

sound_folder = 'sound'
if not os.path.exists(sound_folder):
    os.makedirs(sound_folder) # to create the folder (if it doesn't exist)


<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px">
    Back to the
    <a href="#outilne" style="color: #3d30a2;">
    outline.
    </a>
    <div/>

<div id="1">
   <!-- Contenido de la celda de destino -->
</div>

<h1><span style = "color: rgb(0, 65, 75); font-family: 'PT Serif Pro Book'; color: dark;">1.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">PDF Downloader</span></h1>

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px">
    Our main source for data collection is the <a href="https://www.bcrp.gob.pe/publicaciones/nota-semanal.html" style="color: rgb(0, 153, 123)">BCRP's web page</a>. The weekly note is a periodic (weekly) publication of the BCRP in compliance with article 84 of the Peruvian Constitution and articles 2 and 74 of the BCRP's organic law, which include, among its functions, the periodic publication of the main national macroeconomic statistics. Our project requires the publication of two tables: the table of monthly growth rates of real GDP (12-month percentage changes), and the table of quarterly (annual) growth rates of real GDP. These tables are referred to as Table 1 and Table 2, respectively, throughout this jupyter notebook.
<div/>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    The bot runs the following steps:
    <ol>
        <li>Download the PDF files (NS) from the BCRP web page, starting with the oldest and continuing to the most recent.</li>
        <li>Notify you with a fabulous song each time a certain number of downloads is reached.</li>
        <li>Display a window asking if you want to continue with the downloads. You can stop them at any time.</li>
        <li>Report in detail about the downloaded files. If a file has already been downloaded, you will also be notified.</li>
        <li>Save the raw PDFs to the paths set in the preamble of this Jupyter Notebook.</li>
    </ol>
    Try the bot, it's an adventure!
</div>

In [8]:
# Setting the BCRP URL
bcrp_url = "https://www.bcrp.gob.pe/publicaciones/nota-semanal.html"  # Never replace this URL

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px">
    We import the functions required by the bot to execute the above steps.
    <div/>

In [9]:
from gdp_revisions_datasets_functions import play_sound, random_wait, download_pdf

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px">
    Let's run the code for the bot to do its downloading tasks.
    <div/>

In [14]:
# Initialize pygame
pygame.mixer.init()

# List of available sound files
available_sounds = os.listdir(sound_folder)

# Select a random sound
random_sound = random.choice(available_sounds)

# Full path of the random sound
sound_path = os.path.join(sound_folder, random_sound)

# Load the selected sound
pygame.mixer.music.load(sound_path)

# List to keep track of successfully downloaded files
downloaded_files = []

# Load the list of previously downloaded files if it exists
if os.path.exists(os.path.join(download_record, "downloaded_files.txt")):
    with open(os.path.join(download_record, "downloaded_files.txt"), "r") as f:
        downloaded_files = f.read().splitlines()

# Web driver setup
driver_path = os.environ.get('driver_path')
driver = webdriver.Chrome(executable_path=driver_path)

# Number of downloads per batch
downloads_per_batch = 5
# Total number of downloads
total_downloads = 5

try:
    # Open the test page
    driver.get(bcrp_url)
    print("Site opened successfully")

    # Wait for the container area to be present
    wait = WebDriverWait(driver, 60)
    container_area = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="rightside"]')))

    # Get all the links within the container area
    pdf_links = container_area.find_elements(By.XPATH, './/a')

    # Reverse the order of links
    pdf_links = list(reversed(pdf_links))

    # Initialize download counter
    download_counter = 0

    # Iterate over reversed links and download PDFs in batches
    for pdf_link in pdf_links:
        download_counter += 1

        # Get the file name from the URL
        new_url = pdf_link.get_attribute("href")
        file_name = new_url.split("/")[-1]

        # Check if the file has already been downloaded
        if file_name in downloaded_files:
            print(f"{download_counter}. The file {file_name} has already been downloaded previously. Skipping...")
            continue

        # Try to download the file
        try:
            download_pdf(driver, pdf_link, wait, download_counter, raw_pdf, download_record)

            # Update the list of downloaded files
            downloaded_files.append(file_name)

        except Exception as e:
            print(f"Error downloading the file {file_name}: {str(e)}")

        # If the download count reaches a multiple of batch size, notify
        if download_counter % downloads_per_batch == 0:
            print(f"Batch {download_counter // downloads_per_batch} of {downloads_per_batch} completed")

        # If the download count reaches a multiple of 25, ask the user if they want to continue
        if download_counter % 5 == 0: # after the fifth PDF downloaded, you'll listen a beautiful song
            play_sound()
            user_input = input("Do you want to continue downloading? (Enter 'y' to continue, any other key to stop): ")
            pygame.mixer.music.stop()
            if user_input.lower() != 'y':
                break

        # Random wait before the next iteration
        random_wait(5, 10)

        # If total downloads reached, break out of loop
        if download_counter == total_downloads:
            print(f"All downloads completed ({total_downloads} in total)")
            break

except StaleElementReferenceException:
    print("StaleElementReferenceException occurred. Retrying...")

finally:
    # Close the browser when finished
    driver.quit()


Site opened successfully
1. The file ns-01-2013.pdf has already been downloaded previously. Skipping...
2. The file ns-02-2013.pdf has already been downloaded previously. Skipping...
3. The file ns-03-2013.pdf has already been downloaded previously. Skipping...
4. The file ns-04-2013.pdf has already been downloaded previously. Skipping...
5. The file ns-05-2013.pdf has already been downloaded previously. Skipping...
6. The file ns-06-2013.pdf has already been downloaded previously. Skipping...
7. The file ns-07-2013.pdf has already been downloaded previously. Skipping...
8. The file ns-08-2013.pdf has already been downloaded previously. Skipping...
9. The file ns-09-2013.pdf has already been downloaded previously. Skipping...
10. The file ns-10-2013.pdf has already been downloaded previously. Skipping...
11. The file ns-11-2013.pdf has already been downloaded previously. Skipping...
12. The file ns-12-2013.pdf has already been downloaded previously. Skipping...
13. The file ns-13-2013.

140. The file ns-40-2015.pdf has already been downloaded previously. Skipping...
141. The file ns-41-2015.pdf has already been downloaded previously. Skipping...
142. The file ns-42-2015.pdf has already been downloaded previously. Skipping...
143. The file ns-43-2015.pdf has already been downloaded previously. Skipping...
144. The file ns-44-2015.pdf has already been downloaded previously. Skipping...
145. The file ns-45-2015.pdf has already been downloaded previously. Skipping...
146. The file ns-45-2015.pdf has already been downloaded previously. Skipping...
147. The file ns-46-2015.pdf has already been downloaded previously. Skipping...
148. The file ns-47-2015.pdf has already been downloaded previously. Skipping...
149. The file ns-48-2015.pdf has already been downloaded previously. Skipping...
150. The file ns-01-2016.pdf has already been downloaded previously. Skipping...
151. The file ns-02-2016.pdf has already been downloaded previously. Skipping...
152. The file ns-03-2016.pdf

278. The file ns-32-2018.pdf has already been downloaded previously. Skipping...
279. The file ns-33-2018.pdf has already been downloaded previously. Skipping...
280. The file ns-34-2018.pdf has already been downloaded previously. Skipping...
281. The file ns-35-2018.pdf has already been downloaded previously. Skipping...
282. The file ns-36-2018.pdf has already been downloaded previously. Skipping...
283. The file ns-37-2018.pdf has already been downloaded previously. Skipping...
284. The file ns-38-2018.pdf has already been downloaded previously. Skipping...
285. The file ns-39-2018.pdf has already been downloaded previously. Skipping...
286. The file ns-40-2018.pdf has already been downloaded previously. Skipping...
287. The file ns-41-2018.pdf has already been downloaded previously. Skipping...
288. The file ns-42-2018.pdf has already been downloaded previously. Skipping...
289. The file ns-43-2018.pdf has already been downloaded previously. Skipping...
290. The file ns-44-2018.pdf

401. The file ns-20-2021.pdf has already been downloaded previously. Skipping...
402. The file ns-21-2021.pdf has already been downloaded previously. Skipping...
403. The file ns-22-2021.pdf has already been downloaded previously. Skipping...
404. The file ns-23-2021.pdf has already been downloaded previously. Skipping...
405. The file ns-24-2021.pdf has already been downloaded previously. Skipping...
406. The file ns-25-2021.pdf has already been downloaded previously. Skipping...
407. The file ns-26-2021.pdf has already been downloaded previously. Skipping...
408. The file ns-27-2021.pdf has already been downloaded previously. Skipping...
409. The file ns-28-2021.pdf has already been downloaded previously. Skipping...
410. The file ns-29-2021.pdf has already been downloaded previously. Skipping...
411. The file ns-30-2021.pdf has already been downloaded previously. Skipping...
412. The file ns-31-2021.pdf has already been downloaded previously. Skipping...
413. The file ns-32-2021.pdf

522. The file ns-08-2024.pdf has already been downloaded previously. Skipping...
523. The file ns-09-2024.pdf has already been downloaded previously. Skipping...
524. The file ns-10-2024.pdf has already been downloaded previously. Skipping...
525. The file ns-11-2024.pdf has already been downloaded previously. Skipping...
526. The file ns-12-2024.pdf has already been downloaded previously. Skipping...
527. The file ns-13-2024.pdf has already been downloaded previously. Skipping...
528. The file ns-14-2024.pdf has already been downloaded previously. Skipping...
529. The file ns-15-2024.pdf has already been downloaded previously. Skipping...
530. The file ns-16-2024.pdf has already been downloaded previously. Skipping...
531. The file ns-17-2024.pdf has already been downloaded previously. Skipping...
532. The file ns-18-2024.pdf has already been downloaded previously. Skipping...
533. The file ns-19-2024.pdf has already been downloaded previously. Skipping...
534. The file ns-20-2024.pdf

### Ordenando pdf por años

In [ ]:
import os
import shutil


# Obtener la lista de archivos en el directorio
archivos = os.listdir(raw_pdf)

# Iterar sobre cada archivo
for archivo in archivos:
    # Obtener el año del nombre del archivo
    nombre, extension = os.path.splitext(archivo)
    año = None
    partes_nombre = nombre.split('-')
    for parte in partes_nombre:
        if parte.isdigit() and len(parte) == 4:
            año = parte
            break

    # Si se encontró el año, mover el archivo a la carpeta correspondiente
    if año:
        carpeta_destino = os.path.join(raw_pdf, año)
        # Crear la carpeta si no existe
        if not os.path.exists(carpeta_destino):
            os.makedirs(carpeta_destino)
        # Mover el archivo a la carpeta destino
        shutil.move(os.path.join(raw_pdf, archivo), carpeta_destino)


# Recortando PDFs

In [ ]:
import fitz  # PyMuPDF
import os
import tkinter as tk

# Rutas de directorios

input_pdf_record_txt = 'input_pdf_record.txt'

class PopupWindow(tk.Toplevel):
    def __init__(self, root, message):
        super().__init__(root)
        self.root = root
        self.title("Atención!")
        self.message = message
        self.result = None
        self.configure_window()
        self.create_widgets()

    def configure_window(self):
        self.resizable(False, False)  # Evita cambiar el tamaño de la ventana

    def create_widgets(self):
        self.label = tk.Label(self, text=self.message, wraplength=250)  # Ajusta el texto si es demasiado largo
        self.label.pack(pady=10, padx=10)
        self.btn_frame = tk.Frame(self)
        self.btn_frame.pack(pady=5)
        self.btn_yes = tk.Button(self.btn_frame, text="Sí", command=self.yes)
        self.btn_yes.pack(side=tk.LEFT, padx=5)
        self.btn_no = tk.Button(self.btn_frame, text="No", command=self.no)
        self.btn_no.pack(side=tk.RIGHT, padx=5)

        # Calcula el tamaño de la ventana en función del tamaño del texto
        width = self.label.winfo_reqwidth() + 20
        height = self.label.winfo_reqheight() + 100
        self.geometry(f"{width}x{height}")

    def yes(self):
        self.result = True
        self.destroy()

    def no(self):
        self.result = False
        self.destroy()

def search_keywords(pdf_file, keywords):
    pages_with_keywords = []
    with fitz.open(pdf_file) as doc:
        for page_num in range(doc.page_count):
            page = doc.load_page(page_num)
            text = page.get_text()
            if any(keyword in text for keyword in keywords):
                pages_with_keywords.append(page_num)
    return pages_with_keywords

def trim_pdf(pdf_file, pages):
    if not pages:
        print(f"No se encontraron páginas con palabras clave en {pdf_file}")
        return 0
    
    new_pdf_file = os.path.join(input_pdf, os.path.basename(pdf_file))
    
    with fitz.open(pdf_file) as doc:
        new_doc = fitz.open()
        new_doc.insert_pdf(doc, from_page=0, to_page=0)
        for page_num in pages:
            new_doc.insert_pdf(doc, from_page=page_num, to_page=page_num)
        new_doc.save(new_pdf_file)
    
    num_pages_new_pdf = new_doc.page_count
    print(f"El PDF recortado '{new_pdf_file}' tiene {num_pages_new_pdf} páginas.")

    if num_pages_new_pdf == 5:
        final_doc = fitz.open()
        final_doc.insert_pdf(new_doc, from_page=0, to_page=0)
        final_doc.insert_pdf(new_doc, from_page=1, to_page=1)
        final_doc.insert_pdf(new_doc, from_page=3, to_page=3)
        final_doc.save(new_pdf_file)

        num_pages_new_pdf = final_doc.page_count
        print(f"Solo se conservaron la portada y las páginas con 2 tablas de interés en el PDF recortado '{new_pdf_file}'.")
    else:
        print(f"Se conservaron todas las páginas en el PDF recortado '{new_pdf_file}'.")

    return num_pages_new_pdf

def read_input_pdf_files():
    input_pdf_files_path = os.path.join(input_pdf_record, input_pdf_record_txt)
    if not os.path.exists(input_pdf_files_path):
        return set()
    
    with open(input_pdf_files_path, 'r') as file:
        return set(file.read().splitlines())

def write_input_pdf_files(input_pdf_files):
    input_pdf_files_path = os.path.join(input_pdf_record, input_pdf_record_txt)
    sorted_filenames = sorted(input_pdf_files)  # Sort the filenames
    with open(input_pdf_files_path, 'w') as file:
        for filename in sorted_filenames:
            file.write(filename + '\n')

if __name__ == "__main__":
    keywords = ["ECONOMIC SECTORS"]
    root = tk.Tk()
    root.withdraw()  # Oculta la ventana principal de Tkinter

    input_pdf_files = read_input_pdf_files()
    processing_counter = 1

    for folder in os.listdir(raw_pdf):
        folder_path = os.path.join(raw_pdf, folder)
        if os.path.isdir(folder_path):
            print("Procesando carpeta:", folder)
            num_pdfs_trimmed = 0
            for filename in os.listdir(folder_path):
                if filename.endswith(".pdf"):
                    pdf_file = os.path.join(folder_path, filename)
                    if filename in input_pdf_files:
                        print(f"{processing_counter}. El PDF '{filename}' ya ha sido recortado y guardado en '{input_pdf}'...")
                        processing_counter += 1
                        continue
                    print(f"{processing_counter}. Procesando:", pdf_file)
                    
                    pages_with_keywords = search_keywords(pdf_file, keywords)
                    num_pages_new_pdf = trim_pdf(pdf_file, pages_with_keywords)
                    if num_pages_new_pdf > 0:
                        num_pdfs_trimmed += 1
                        input_pdf_files.add(filename)
                        processing_counter += 1
            
            write_input_pdf_files(input_pdf_files)

            message = f"{num_pdfs_trimmed} PDFs han sido recortados en la carpeta {folder}. ¿Desea continuar?"
            popup = PopupWindow(root, message)
            root.wait_window(popup)
            if not popup.result:
                break
                
    print("Proceso completado para todos los PDFs en el directorio:", input_pdf)


### Ordenando pdf por años

In [ ]:
import os
import shutil


# Obtener la lista de archivos en el directorio
archivos = os.listdir(input_pdf)

# Iterar sobre cada archivo
for archivo in archivos:
    # Obtener el año del nombre del archivo
    nombre, extension = os.path.splitext(archivo)
    año = None
    partes_nombre = nombre.split('-')
    for parte in partes_nombre:
        if parte.isdigit() and len(parte) == 4:
            año = parte
            break

    # Si se encontró el año, mover el archivo a la carpeta correspondiente
    if año:
        carpeta_destino = os.path.join(input_pdf, año)
        # Crear la carpeta si no existe
        if not os.path.exists(carpeta_destino):
            os.makedirs(carpeta_destino)
        # Mover el archivo a la carpeta destino
        shutil.move(os.path.join(input_pdf, archivo), carpeta_destino)


<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px">
    Back to the
    <a href="#outilne" style="color: #3d30a2;">
    outline.
    </a>
    <div/>

<div id="2">
   <!-- Contenido de la celda de destino -->
</div>

<h1><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;; color: dark;">2.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Data cleaning</span></h1>

<div id="2-2">
   <!-- Contenido de la celda de destino -->
</div>

<h2><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">2.2.</span>
    <span style = "color: dark; font-family: PT Serif Pro Book;">
    Extracting tables and generating dataframes (includes data cleanup)
    </span>
    </h2>

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px">
    We would like to get specific tables: information on GDP growth rates with annual, quarterly and monthly frequency. We don't need other tables also related to GDP that don't meet these requirements. Extraction will be easier if we use keywords.
    <div/>

In [ ]:
# Keywords to search in the page text
keywords = ["PRODUCTO BRUTO INTERNO", "SECTORES ECONÓMICOS", "PBI", "GDP", "Variaciones"]

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px">
    The code iterates through each PDF and extracts the two required tables from each. The extracted information is then transformed into dataframes and the columns and values are cleaned up to conform to Python conventions (pythonic).
    <div/>

# Funciones

### Auxiliares

In [ ]:
def remove_rare_characters_first_row(texto):
    texto = re.sub(r'\s*-\s*', '-', texto)  # Remueve espacios alrededor de guiones
    texto = re.sub(r'[^a-zA-Z0-9\s-]', '', texto)  # Remueve caracteres raros excepto letras, dígitos y guiones
    return texto

def remove_rare_characters(texto):
    return re.sub(r'[^a-zA-Z\s]', '', texto)

def remove_tildes(texto):
    return ''.join((c for c in unicodedata.normalize('NFD', texto) if unicodedata.category(c) != 'Mn'))

### Común

In [ ]:
# 0.
def drop_nan_rows(df):
    df = df.dropna(how='all')
    return df

# 1. 
def drop_nan_columns(df):
    return df.dropna(axis=1, how='all')

# 2.
def swap_first_second_row(df):
    temp = df.iloc[0, 0]
    df.iloc[0, 0] = df.iloc[1, 0]
    df.iloc[1, 0] = temp

    temp = df.iloc[0, -1]
    df.iloc[0, -1] = df.iloc[1, -1]
    df.iloc[1, -1] = temp
    return df

# 8. 
def reset_index(df):
    df.reset_index(drop=True, inplace=True)
    return df

# 5.
def remove_digit_slash(df):
    # Aplica la función de reemplazo a la primera columna y a las dos últimas columnas
    df.iloc[:, [0, -2, -1]] = df.iloc[:, [0, -2, -1]].apply(lambda x: x.str.replace(r'\d+/', '', regex=True))
    return df

# 9. AUX (ROBUSTO)

def separate_text_digits(df):
    for index, row in df.iterrows():
        if any(char.isdigit() for char in str(row.iloc[-2])) and any(char.isalpha() for char in str(row.iloc[-2])):
            if pd.isnull(row.iloc[-1]):
                df.loc[index, df.columns[-1]] = ''.join(filter(lambda x: x.isalpha() or x == ' ', str(row.iloc[-2])))
                df.loc[index, df.columns[-2]] = ''.join(filter(lambda x: not (x.isalpha() or x == ' '), str(row.iloc[-2])))
            
            # Check if comma or dot is used as decimal separator
            if ',' in str(row.iloc[-2]):
                split_values = str(row.iloc[-2]).split(',')
            elif '.' in str(row.iloc[-2]):
                split_values = str(row.iloc[-2]).split('.')
            else:
                # If neither comma nor dot found, assume no decimal part
                split_values = [str(row.iloc[-2]), '']
                
            cleaned_integer = ''.join(filter(lambda x: x.isdigit() or x == '-', split_values[0]))
            cleaned_decimal = ''.join(filter(lambda x: x.isdigit(), split_values[1]))
            if cleaned_decimal:
                # Use comma as decimal separator
                cleaned_numeric = cleaned_integer + ',' + cleaned_decimal
            else:
                cleaned_numeric = cleaned_integer
            df.loc[index, df.columns[-2]] = cleaned_numeric
    return df


# 4. 
def extract_years(df):
    year_columns = [col for col in df.columns if re.match(r'\b\d{4}\b', col)]
    #print("Años (4 dígitos) extraídos:")
    #print(year_columns)
    return year_columns

# 6. 
def first_row_columns(df):
    df.columns = df.iloc[0]
    df = df.drop(df.index[0])
    return df

# 15.
def clean_columns_values(df):
    df.columns = df.columns.str.lower()
    # Only normalize string column names
    df.columns = [unicodedata.normalize('NFKD', col).encode('ASCII', 'ignore').decode('utf-8') if isinstance(col, str) else col for col in df.columns]
    df.columns = df.columns.str.replace(' ', '_').str.replace('ano', 'year').str.replace('-', '_')
    
    text_columns = df.select_dtypes(include='object').columns
    for col in df.columns:
        df.loc[:, col] = df[col].apply(lambda x: remove_tildes(x) if isinstance(x, str) else x)
        df.loc[:, col] = df[col].apply(lambda x: str(x).replace(',', '.') if isinstance(x, (int, float, str)) else x)
    df.loc[:, 'sectores_economicos'] = df['sectores_economicos'].str.lower()
    df.loc[:, 'economic_sectors'] = df['economic_sectors'].str.lower()
    df.loc[:, 'sectores_economicos'] = df['sectores_economicos'].apply(remove_rare_characters)
    df.loc[:, 'economic_sectors'] = df['economic_sectors'].apply(remove_rare_characters)
    return df

# 16.
def convertir_float(df):
    excluded_columns = ['sectores_economicos', 'economic_sectors']
    columns_to_convert = [col for col in df.columns if col not in excluded_columns]
    df[columns_to_convert] = df[columns_to_convert].apply(pd.to_numeric, errors='coerce')
    return df

# 15.
def relocate_last_column(df):
    last_column = df.pop(df.columns[-1])
    df.insert(1, last_column.name, last_column)
    return df

### Exclusiva Tabla 1

In [ ]:
# ATIPIC LAST COLUMNS
def relocate_last_columns(df):
    if not pd.isna(df.iloc[1, -1]):
        # Create a new column with NaN
        new_column = 'col_' + ''.join(map(str, np.random.randint(1, 5, size=1)))
        df[new_column] = np.nan
        
        # Get 'ECONOMIC SECTORS' and relocate
        insert_value_1 = df.iloc[0, -2]
        # Convert the value to string before assignment
        insert_value_1 = str(insert_value_1)
        # Ensure the dtype of the last column is object (string) to accommodate string values
        df.iloc[:, -1] = df.iloc[:, -1].astype('object')
        df.iloc[0, -1] = insert_value_1
        
        # NaN first obs
        df.iloc[0,-2] = np.nan
    return df

# Extraer meses

def get_months_sublist_list(df, year_columns):
    first_row = df.iloc[0]
    # Initialize the list of sublists
    months_sublist_list = []

    # Initialize the current sublist
    months_sublist = []

    # Iterate over the elements of the first row
    for item in first_row:
        # Check if the item meets the requirements
        if len(str(item)) == 3:
            months_sublist.append(item)
        elif '-' in item or str(item) == 'year':
            months_sublist.append(item)
            months_sublist_list.append(months_sublist)
            months_sublist = []

    # Add the last sublist if it's not empty
    if months_sublist:
        months_sublist_list.append(months_sublist)

    new_elements = []

    # Check if year_columns is not empty
    if year_columns:
        for i, year in enumerate(year_columns):
            # Check if index i is valid for quarters_sublist_list
            if i < len(months_sublist_list):
                for element in months_sublist_list[i]:
                    new_elements.append(f"{year}_{element}")
                    
    two_first_elements = df.iloc[0][:2].tolist()

    # Ensure that the two_first_elements are added if they are not in new_elements
    for index in range(len(two_first_elements) - 1, -1, -1):
        if two_first_elements[index] not in new_elements:
            new_elements.insert(0, two_first_elements[index])

    # Ensure that the length of new_elements matches the number of columns in df
    while len(new_elements) < len(df.columns):
        new_elements.append(None)

    temp_df = pd.DataFrame([new_elements], columns=df.columns)
    df.iloc[0] = temp_df.iloc[0]

    return df


def find_year_column(df):
    # List to store the found years
    found_years = []

    # Iterating over the column names of the DataFrame
    for column in df.columns:
        # Checking if the column name is a year (4 digits)
        if column.isdigit() and len(column) == 4:
            found_years.append(column)

    # If more than one year is found, do nothing
    if len(found_years) > 1:
        pass
    # If exactly one year is found, implement additional code
    elif len(found_years) == 1:
        # Getting the name of the found year
        year_name = found_years[0]
        print("The name of the column representing the year is:", year_name)

        # Getting the first row of the DataFrame
        first_row = df.iloc[0]

        # Searching for the first column containing the word "year" or some hyphen-separated expression
        column_contains_year = first_row[first_row.astype(str).str.contains(r'\byear\b')]

        if not column_contains_year.empty:
            # Getting the name of the first column containing 'year' or some hyphen-separated expression in the first row
            column_contains_year_name = column_contains_year.index[0]
            print("The name of the first column containing 'year' or some hyphen-separated expression in the first row is:", column_contains_year_name)

            # Getting the indices of the columns
            column_contains_year_index = df.columns.get_loc(column_contains_year_name)
            year_name_index = df.columns.get_loc(year_name)
            print("The index of the column containing 'year' is:", column_contains_year_index)
            print("The index of the column representing the year is:", year_name_index)

            # Checking if the column representing the year is to the right or to the left of column_contains_year
            if column_contains_year_index < year_name_index:
                print("The year column is to the right of the column containing 'year'.")
                # Adding one to the year
                new_year = str(int(year_name) - 1)
                # Renaming the column containing 'year' with the new year
                df.rename(columns={column_contains_year_name: new_year}, inplace=True)
                print(f"The column containing 'year' is now named '{new_year}'.")
            elif column_contains_year_index > year_name_index:
                print("The year column is to the left of the column containing 'year'.")
                # Subtracting one from the year
                new_year = str(int(year_name) + 1)
                # Renaming the year column with the new year
                df.rename(columns={column_contains_year_name: new_year}, inplace=True)
                print(f"The column containing 'year' is now named '{new_year}'.")
            else:
                print("The year column is in the same position as the column containing 'year'.")
        else:
            print("No columns containing 'year' were found in the first row.")
    # If no year is found, print a message
    else:
        print("No years were found in the column names.")
    
    return df


#

def clean_first_row(df):
    for col in df.columns:
        if df[col].dtype == 'object':
            if isinstance(df.at[0, col], str):
                df.at[0, col] = df.at[0, col].lower()  # Convertir a minúsculas solo si es un objeto
                df.at[0, col] = remove_tildes(df.at[0, col])
                df.at[0, col] = remove_rare_characters_first_row(df.at[0, col])
                # Reemplazar 'ano' por 'year'
                df.at[0, col] = df.at[0, col].replace('ano', 'year')

    return df


def intercambiar_valores(df):
    # Verificar si hay al menos dos columnas en el DataFrame
    if len(df.columns) < 2:
        print("El DataFrame tiene menos de dos columnas. No se pueden intercambiar valores.")
        return df

    # Verificar si hay valores NaN en la última columna
    if df.iloc[:, -1].isnull().any():
        # Obtener índice de filas con NaN en la última columna
        last_column_rows_nan = df[df.iloc[:, -1].isnull()].index

        # Iterar sobre las filas con NaN en la última columna
        for idx in last_column_rows_nan:
            # Verificar si el índice está dentro del rango de las columnas
            if -2 >= -len(df.columns):
                # Intercambiar los valores de la última columna y la penúltima columna
                df.iloc[idx, -1], df.iloc[idx, -2] = df.iloc[idx, -2], df.iloc[idx, -1]
            else:
                print(f"Índice fuera de rango para la fila {idx}. No se pueden intercambiar valores.")

    return df



def replace_var_perc_first_column(df):
    # Regular expression to search for "Var. %" or "Var.%"
    regex = re.compile(r'Var\. ?%')

    # Iterate over the rows of the dataframe
    for index, row in df.iterrows():
        # Convert the value in the first column to a string
        value = str(row.iloc[0])

        # Check if the value matches the regular expression
        if regex.search(value):
            # Replace only the characters that match the regular expression
            df.at[index, df.columns[0]] = regex.sub("variacion porcentual", value)
    
    return df


# 8.
number_moving_average = 'three' # Keep a space at the end

def replace_number_moving_average(df):
    for index, row in df.iterrows():
        # Buscar la expresión regular en la penúltima o última columna
        if pd.notnull(row.iloc[-1]) and re.search(r'(\d\s*-)', str(row.iloc[-1])):
            df.at[index, df.columns[-1]] = re.sub(r'(\d\s*-)', f'{number_moving_average}-', str(row.iloc[-1]))
        #elif pd.notnull(row.iloc[-2]) and re.search(r'(\d\s*-)', str(row.iloc[-2])):
        #   df.at[index, df.columns[-2]] = re.sub(r'(\d\s*-)', f'{number_moving_average}-', str(row.iloc[-2]))
    return df


# 7.
def replace_var_perc_last_columns(df):
    # Expresión regular para buscar "Var. %" o "Var.%"
    regex = re.compile(r'(Var\. ?%)(.*)')

    # Iterar sobre las filas del dataframe
    for index, row in df.iterrows():
        # Verificar si el valor en la penúltima columna es una cadena no nula
        if isinstance(row.iloc[-2], str) and regex.search(row.iloc[-2]):
            # Realizar el reemplazo al final del valor de la penúltima columna
            replaced_text = regex.sub(r'\2 percent change', row.iloc[-2])
            df.at[index, df.columns[-2]] = replaced_text.strip()
        
        # Verificar si el valor en la última columna es una cadena no nula
        if isinstance(row.iloc[-1], str) and regex.search(row.iloc[-1]):
            # Realizar el reemplazo al final del valor de la última columna
            replaced_text = regex.sub(r'\2 percent change', row.iloc[-1])
            df.at[index, df.columns[-1]] = replaced_text.strip()
    
    return df

# Función para buscar y reemplazar en la segunda fila del DataFrame
def replace_first_dot(df):
    second_row = df.iloc[1]  # Segunda fila del DataFrame
    
    # Verificar si al menos una observación cumple con el patrón
    if any(isinstance(cell, str) and re.match(r'^\w+\.\s?\w+', cell) for cell in second_row):
        for col in df.columns:
            if isinstance(second_row[col], str):  # Verificar si el valor es una cadena
                if re.match(r'^\w+\.\s?\w+', second_row[col]):  # Verificar si cumple con el patrón Xxx.Xxx o Xxx. Xxx.
                    df.at[1, col] = re.sub(r'(\w+)\.(\s?\w+)', r'\1-\2', second_row[col], count=1)  # Reemplazar solo el primer punto
    return df

def drop_rare_caracter_row(df):
    # Buscar el caracter solitario "}" en cada fila y obtener un booleano para cada fila
    rare_caracter_row = df.apply(lambda row: '}' in row.values, axis=1)
    
    # Filtrar el DataFrame para eliminar las filas con el caracter solitario "}"
    df = df[~rare_caracter_row]
    
    return df

def split_column_by_pattern(df):
    # Iteramos sobre las columnas del dataframe
    for col in df.columns:
        # Verificamos si la segunda fila de la columna contiene el patrón
        if re.match(r'^[A-Z][a-z]+\.?\s[A-Z][a-z]+\.?$', str(df.iloc[1][col])):
            # Realizamos el split de la columna usando como criterio el espacio
            split_values = df[col].str.split(expand=True)
            # Guardamos los primeros valores en la columna original
            df[col] = split_values[0]
            # Guardamos los segundos valores en una nueva columna con el sufijo "_split"
            new_col_name = col + '_split'
            df.insert(df.columns.get_loc(col) + 1, new_col_name, split_values[1])
    return df

$\Large{\color{blue}{ns\_2014\_07}}$

se: sectores económicos

In [ ]:
def swap_nan_se(df):
    # Check if the first observation of the first column is NaN
    if pd.isna(df.iloc[0, 0]) and df.iloc[0, 1] == "SECTORES ECONÓMICOS":
        # Create a temporary copy of the values
        column_1_value = df.iloc[0, 1]
        # Swap values in the original row
        df.iloc[0, 0] = column_1_value
        df.iloc[0, 1] = np.nan
        # Drop the second column
        df = df.drop(df.columns[1], axis=1)
    return df

$\Large{\color{blue}{ns\_2014\_08}}$

In [ ]:
def replace_first_row_with_columns(df):
    # Check if the first row contains at least one year
    if any(isinstance(element, str) and element.isdigit() and len(element) == 4 for element in df.iloc[0]):
        # Replace NaN values in the first row with random column names
        for col_index, value in enumerate(df.iloc[0]):
            if pd.isna(value):
                df.iloc[0, col_index] = f"column_{col_index + 1}"
        # Replace column names with the values of the first row
        df.columns = df.iloc[0]
        # Drop the first row after setting it as column names
        df = df.drop(df.index[0])
    return df

In [ ]:
def expand_column(df):
    columna_a_expandir = df.columns[-2]
    
    def reemplazar_guiones(match_obj):
        return match_obj.group(1) + ' ' + match_obj.group(2)    

    if df[columna_a_expandir].str.contains(r'\d').any() and df[columna_a_expandir].str.contains(r'[a-zA-Z]').any():
        df[columna_a_expandir] = df[columna_a_expandir].apply(lambda x: re.sub(r'([a-zA-Z]+)\s*-\s*([a-zA-Z]+)', reemplazar_guiones, str(x)) if pd.notnull(x) else x)

        
        # Expresión regular para extraer palabras
        pattern = re.compile(r'[a-zA-Z\s]+$')

        # Función para aplicar la lógica de extracción y reemplazo a cada fila
        def extract_replace(row):
            if pd.notnull(row[columna_a_expandir]) and isinstance(row[columna_a_expandir], str):  # Verifica que el valor no sea NaN y sea de tipo string
                if row.name != 0:  # Para que empiece desde la segunda fila
                    value_to_replace = pattern.search(row[columna_a_expandir])
                    if value_to_replace:
                        value_to_replace = value_to_replace.group().strip()
                        row[df.columns[-1]] = value_to_replace
                        row[columna_a_expandir] = re.sub(pattern, '', row[columna_a_expandir]).strip()
            return row

        # Aplicar la función a cada fila del DataFrame
        df = df.apply(extract_replace, axis=1)

    return df

In [ ]:
def split_values_1(df):
    columna_a_expandir = df.columns[-2]
    nuevas_columnas = df[columna_a_expandir].str.split(expand=True)
    nuevas_columnas.columns = [f'{columna_a_expandir}_{i+1}' for i in range(nuevas_columnas.shape[1])]
    posicion_insercion = len(df.columns) - 1
    for col in reversed(nuevas_columnas.columns):
        df.insert(posicion_insercion, col, nuevas_columnas[col])
    df.drop(columns=[columna_a_expandir], inplace=True)
    return df

$\Large{\color{blue}{ns\_2015\_11}}$

In [ ]:
def revisar_primera_fila(df):
    primera_fila = df.iloc[0]  # Obtenemos la primera fila del DataFrame
    
    for i, (col, valor) in enumerate(primera_fila.items()):  # Iteramos sobre los índices y valores de la primera fila
        # Comprobamos si algún valor de la primera fila tiene dos años juntos
        if re.search(r'\b\d{4}\s\d{4}\b', str(valor)):
            # Si es así, extraemos los dos años
            anios = valor.split()
            primer_anio = anios[0]
            segundo_anio = anios[1]
            
            # Nombre de la columna original
            nombre_columna_original = f'col_{i}'
            df.at[0, col] = nombre_columna_original
            
            # Actualizamos el valor de la primera columna si es NaN con el primer año
            if pd.isna(df.iloc[0, 0]):
                df.iloc[0, 0] = primer_anio
            
            # Actualizamos el valor de la segunda columna si es NaN con el segundo año
            if pd.isna(df.iloc[0, 1]):
                df.iloc[0, 1] = segundo_anio
    
    return df

In [ ]:
def replace_nan_with_previous_column_3(df):
    columns = df.columns
    
    for i in range(len(columns) - 1):
        # Añadir condición para verificar si la columna actual no es la última
        if i != len(columns) - 1 and (columns[i].endswith('_year') and not df[columns[i]].isnull().any()):
            # Revisar si la columna de la derecha tiene al menos un NaN y no termina en '_year'
            if df[columns[i+1]].isnull().any() and not columns[i+1].endswith('_year'):
                nan_indices = df[columns[i+1]].isnull()
                df.loc[nan_indices, [columns[i], columns[i+1]]] = df.loc[nan_indices, [columns[i+1], columns[i]]].values
    
    return df

$\Large{\color{blue}{ns\_2016\_15}}$

In [ ]:
def revisar_primera_fila_1(df):
    # Comprobar si el valor en la primera fila y primera columna es NaN
    if pd.isnull(df.iloc[0, 0]):
        # Comprobar si el valor en la penúltima columna y primera fila es un año (4 dígitos)
        penultimate_column = df.iloc[0, -2]
        if isinstance(penultimate_column, str) and len(penultimate_column) == 4 and penultimate_column.isdigit():
            # Intercambiar los valores
            df.iloc[0, 0] = penultimate_column
            df.iloc[0, -2] = np.nan
    
    # Comprobar si el valor en la segunda columna y primera fila es NaN
    if pd.isnull(df.iloc[0, 1]):
        # Comprobar si el valor en la última columna y primera fila es un año (4 dígitos)
        last_column = df.iloc[0, -1]
        if isinstance(last_column, str) and len(last_column) == 4 and last_column.isdigit():
            # Intercambiar los valores
            df.iloc[0, 1] = last_column
            df.iloc[0, -1] = np.nan
    
    return df

In [ ]:
def split_values_2(df):
    columna_a_expandir = df.columns[-4]
    nuevas_columnas = df[columna_a_expandir].str.split(expand=True)
    nuevas_columnas.columns = [f'{columna_a_expandir}_{i+1}' for i in range(nuevas_columnas.shape[1])]
    posicion_insercion = len(df.columns) - 3
    for col in reversed(nuevas_columnas.columns):
        df.insert(posicion_insercion, col, nuevas_columnas[col])
    df.drop(columns=[columna_a_expandir], inplace=True)
    return df

### Exclusiva Tabla 2

In [ ]:
def clean_first_row(df):
    for col in df.columns:
        if df[col].dtype == 'object':
            if isinstance(df.at[0, col], str):
                df.at[0, col] = df.at[0, col].lower()  # Convertir a minúsculas solo si es un objeto
                df.at[0, col] = remove_tildes(df.at[0, col])
                df.at[0, col] = remove_rare_characters_first_row(df.at[0, col])
                # Reemplazar 'ano' por 'year'
                df.at[0, col] = df.at[0, col].replace('ano', 'year')

    return df

# 2.
def separate_years(df):
    df = df.copy()  # Se crea una copia del DataFrame para evitar SettingWithCopyWarning
    if isinstance(df.iloc[0, -2], str) and len(df.iloc[0, -2].split()) == 2:
        years = df.iloc[0, -2].split()
        if all(len(year) == 4 for year in years):
            segundo_anio = years[1]
            df.iloc[0, -2] = years[0]
            df.insert(len(df.columns) - 1, 'new_column', [segundo_anio] + [None] * (len(df) - 1))
    return df

# 3.
def find_roman_numerals(text):
    pattern = r'\b(?:I{1,3}|IV|V|VI{0,3}|IX|X)\b'
    matches = re.findall(pattern, text)
    return matches

def relocate_roman_numerals(df):
    numeros_romanos = find_roman_numerals(df.iloc[2, -1])
    if numeros_romanos:
        original_text = df.iloc[2, -1]
        for roman_numeral in numeros_romanos:
            original_text = original_text.replace(roman_numeral, '').strip()
        df.iloc[2, -1] = original_text
        df.at[2, 'new_column'] = ', '.join(numeros_romanos)
        df.iloc[2, -1] = np.nan
    return df

# 4.
def extract_mixed_values(df):
    df = df.copy()  # Se crea una copia del DataFrame para evitar SettingWithCopyWarning
    regex_pattern = r'(-?\d+,\d [a-zA-Z\s]+)'
    for index, row in df.iterrows():
        antepenultima_obs = row.iloc[-3]
        penultima_obs = row.iloc[-2]

        if isinstance(antepenultima_obs, str) and pd.notnull(antepenultima_obs):
            match = re.search(regex_pattern, antepenultima_obs)
            if match:
                parte_extraida = match.group(0)
                if pd.isna(penultima_obs) or pd.isnull(penultima_obs):
                    df.iloc[index, -2] = parte_extraida
                    antepenultima_obs = re.sub(regex_pattern, '', antepenultima_obs).strip()
                    df.iloc[index, -3] = antepenultima_obs
    return df

# 5.
def replace_first_row_nan(df):
    for col in df.columns:
        if pd.isna(df.iloc[0][col]):
            df.iloc[0, df.columns.get_loc(col)] = col
    return df

# 11. 
def split_values(df):
    columna_a_expandir = df.columns[-3]
    nuevas_columnas = df[columna_a_expandir].str.split(expand=True)
    nuevas_columnas.columns = [f'{columna_a_expandir}_{i+1}' for i in range(nuevas_columnas.shape[1])]
    posicion_insercion = len(df.columns) - 2
    for col in reversed(nuevas_columnas.columns):
        df.insert(posicion_insercion, col, nuevas_columnas[col])
    df.drop(columns=[columna_a_expandir], inplace=True)
    return df


# 13.
def roman_arabic(df):
    primera_fila = df.iloc[0]
    def convert_roman_number(numero):
        try:
            return str(roman.fromRoman(numero))
        except roman.InvalidRomanNumeralError:
            return numero

    primera_fila_convertida = []
    for valor in primera_fila:
        if isinstance(valor, str) and not pd.isna(valor):
            primera_fila_convertida.append(convert_roman_number(valor))
        else:
            primera_fila_convertida.append(valor)

    df.iloc[0] = primera_fila_convertida
    return df

# 14.
def fix_duplicates(df):
    fila_segunda = df.iloc[0].copy()
    prev_num = None
    first_one_index = None

    for i, num in enumerate(fila_segunda):
        try:
            num = int(num)
            prev_num = int(prev_num) if prev_num is not None else None

            if num == prev_num:
                if num == 1:
                    if first_one_index is None:
                        first_one_index = i - 1
                    next_num = int(fila_segunda[i - 1]) + 1
                    for j in range(i, len(fila_segunda)):
                        if fila_segunda.iloc[j].isdigit():
                            fila_segunda.iloc[j] = str(next_num)
                            next_num += 1
                elif i - 1 >= 0:
                    fila_segunda.iloc[i] = str(int(fila_segunda.iloc[i - 1]) + 1)

            prev_num = num
        except ValueError:
            pass

    df.iloc[0] = fila_segunda
    return df

## More

In [ ]:
def get_quarters_sublist_list(df, year_columns):
    first_row = df.iloc[0]
    # Initialize the list of sublists
    quarters_sublist_list = []

    # Initialize the current sublist
    quarters_sublist = []

    # Iterate over the elements of the first row
    for item in first_row:
        # Check if the item meets the requirements
        if len(str(item)) == 1:
            quarters_sublist.append(item)
        elif str(item) == 'year':
            quarters_sublist.append(item)
            quarters_sublist_list.append(quarters_sublist)
            quarters_sublist = []

    # Add the last sublist if it's not empty
    if quarters_sublist:
        quarters_sublist_list.append(quarters_sublist)

    new_elements = []

    # Check if year_columns is not empty
    if year_columns:
        for i, year in enumerate(year_columns):
            # Check if index i is valid for quarters_sublist_list
            if i < len(quarters_sublist_list):
                for element in quarters_sublist_list[i]:
                    new_elements.append(f"{year}_{element}")

    two_first_elements = df.iloc[0][:2].tolist()

    # Ensure that the two_first_elements are added if they are not in new_elements
    for index in range(len(two_first_elements) - 1, -1, -1):
        if two_first_elements[index] not in new_elements:
            new_elements.insert(0, two_first_elements[index])

    # Ensure that the length of new_elements matches the number of columns in df
    while len(new_elements) < len(df.columns):
        new_elements.append(None)

    temp_df = pd.DataFrame([new_elements], columns=df.columns)
    df.iloc[0] = temp_df.iloc[0]

    return df


$\Large{\color{blue}{ns\_2016\_19}}$

In [ ]:
def split_values_3(df):
    columna_a_expandir = df.columns[-3]
    nuevas_columnas = df[columna_a_expandir].str.split(expand=True)
    nuevas_columnas.columns = [f'{columna_a_expandir}_{i+1}' for i in range(nuevas_columnas.shape[1])]
    posicion_insercion = len(df.columns) - 2
    for col in reversed(nuevas_columnas.columns):
        df.insert(posicion_insercion, col, nuevas_columnas[col])
    df.drop(columns=[columna_a_expandir], inplace=True)
    return df

In [ ]:
def replace_nan_with_previous_column_1(df):
    columns = df.columns
    
    for i in range(len(columns) - 1):
        # Añadir condición para verificar si la columna actual no es la última
        if i != len(columns) - 2 and not (columns[i].endswith('_year') and df[columns[i]].isnull().any()):
            # Revisar si la columna de la derecha tiene al menos un NaN y no termina en '_year'
            if df[columns[i+1]].isnull().any() and not columns[i+1].endswith('_year'):
                nan_indices = df[columns[i+1]].isnull()
                df.loc[nan_indices, [columns[i], columns[i+1]]] = df.loc[nan_indices, [columns[i+1], columns[i]]].values
    
    return df

In [ ]:
def replace_nan_with_previous_column_2(df):
    columns = df.columns
    
    for i in range(len(columns) - 1):
        # Añadir condición para verificar si la columna actual no es la última
        if i != len(columns) - 2 and not (columns[i].endswith('_year') and df[columns[i]].isnull().any()):
            # Revisar si la columna de la derecha tiene al menos un NaN y no termina en '_year'
            if df[columns[i+1]].isnull().any() and not columns[i+1].endswith('_year'):
                nan_indices = df[columns[i+1]].isnull()
                df.loc[nan_indices, [columns[i], columns[i+1]]] = df.loc[nan_indices, [columns[i+1], columns[i]]].values
    
    return df

$\Large{\color{blue}{ns\_2016\_20}}$

In [ ]:
def drop_nan_row(df):
    if df.iloc[0].isnull().all():
        df = df.drop(index=0)
        df.reset_index(drop=True, inplace=True)
    return df

$\Large{\color{blue}{ns\_2019\_17}}$

In [ ]:
def last_column_es(df): # similar than relocate last columns
    # Check if the DataFrame has at least two columns and the last column is a 4-digit year
    #if len(df.columns) >= 2 and df.columns[-1].isdigit() and len(df[df.columns[-1]].iloc[:2]) >= 2:
    # Check if the first observation of the last column is 'ECONOMIC SECTORS'
    if df[df.columns[-1]].iloc[0] == 'ECONOMIC SECTORS':
        # Check if the second observation of the last column is not empty
        if pd.notnull(df[df.columns[-1]].iloc[1]):
            # Create a new column with NaN values
            new_column_name = f"col_{len(df.columns)}"
            df[new_column_name] = np.nan

            # Get 'ECONOMIC SECTORS' and relocate
            insert_value_1 = df.iloc[0, -2]
            # Convert the value to string before assignment
            insert_value_1 = str(insert_value_1)
            # Ensure the dtype of the last column is object (string) to accommodate string values
            df.iloc[:, -1] = df.iloc[:, -1].astype('object')
            df.iloc[0, -1] = insert_value_1

            # NaN first obs
            df.iloc[0,-2] = np.nan
    return df

$\Large{\color{blue}{ns\_2019\_26}}$

In [ ]:
def intercambiar_columnas(df):
    # Buscar una columna con todos los valores NaN
    columna_nan = None
    for columna in df.columns:
        if df[columna].isnull().all() and len(columna) == 4 and columna.isdigit():
            columna_nan = columna
            break
    
    if columna_nan:
        # Revisar la columna de la izquierda
        indice_columna = df.columns.get_loc(columna_nan)
        if indice_columna > 0:
            columna_izquierda = df.columns[indice_columna - 1]
            # Verificar si no es un año (no tiene 4 dígitos)
            if not (len(columna_izquierda) == 4 and columna_izquierda.isdigit()):
                # Intercambiar nombres de columnas
                df.rename(columns={columna_nan: columna_izquierda, columna_izquierda: columna_nan}, inplace=True)
    
    return df

$\Large{\color{blue}{ns\_2019\_29}}$

In [ ]:
def exchange_roman_nan(df):
    for col_idx, valor in enumerate(df.iloc[1]):
        if isinstance(valor, str):
            if valor.upper() == 'AÑO' or (valor.isalpha() and roman.fromRoman(valor.upper())):
                siguiente_col = col_idx + 1
                if siguiente_col < len(df.columns) and pd.isna(df.iloc[1, siguiente_col]):
                    col_actual = df.iloc[:, col_idx].drop(index=1)
                    col_siguiente = df.iloc[:, siguiente_col].drop(index=1)
                    if col_actual.isna().all():
                        df.iloc[1, col_idx], df.iloc[1, siguiente_col] = df.iloc[1, siguiente_col], df.iloc[1, col_idx]
    return df

$\Large{\color{blue}{set \ by \ sep}}$

In [ ]:
def replace_set_sep(df):
    # Get the column names of the DataFrame
    columns = df.columns
    
    # Iterate over the columns
    for column in columns:
        # Check if the column contains the expression 'set'
        if 'set' in column:
            # Replace 'set' with 'sep' in the column name
            new_column = column.replace('set', 'sep')
            # Rename the column in the DataFrame
            df.rename(columns={column: new_column}, inplace=True)
    
    return df

$\Large{\color{blue}{strip}}$

In [ ]:
def spaces_se_es(df):
    # Aplicar strip a las columnas 'sectores_economicos' y 'economic_sectors'
    df['sectores_economicos'] = df['sectores_economicos'].str.strip()
    df['economic_sectors'] = df['economic_sectors'].str.strip()
    return df

$\Large{\color{blue}{Replace \ other \ services}}$

In [ ]:
def replace_services(df):
    # Verificamos si se encuentran los valores 'otros servicios' y 'other services'
    if ('servicios' in df['sectores_economicos'].values) and ('services' in df['economic_sectors'].values):
        # Reemplazamos los valores
        df['sectores_economicos'].replace({'servicios': 'otros servicios'}, inplace=True)
        df['economic_sectors'].replace({'services': 'other services'}, inplace=True)
    return df

$\Large{\color{blue}{Redondear \ a \ un \ decimal}}$

In [ ]:
def redondear_valores(df, decimales=1):
    # Iterar sobre todas las columnas del DataFrame
    for col in df.columns:
        # Verificar si la columna es de tipo float
        if df[col].dtype == 'float64':
            # Redondear los valores de la columna al número de decimales especificado
            df[col] = df[col].round(decimales)
    return df

# Table 1

# Con registro de carpetas procesdas 1

In [ ]:
import tabula
import pdfplumber
import pandas as pd
import numpy as np
import os
import re
from datetime import datetime
import locale
from tkinter import Tk, messagebox, TOP, YES, NO
import os
from sqlalchemy import create_engine

# Establecer la localización en español
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

# Palabras clave para buscar en el texto de la página
keywords = ["ECONOMIC SECTORS"]

# Diccionario para almacenar los DataFrames generados
dataframes_dict_1 = {}

# Ruta del archivo de registro de carpetas procesadas
registro_path = 'dataframes_record/carpetas_procesadas_1.txt'

# Función para corregir los nombres de los meses
def corregir_nombre_mes(mes):
    meses_mapping = {
        'setiembre': 'septiembre',
        # Agrega más mapeos si es necesario para otros nombres de meses
    }
    return meses_mapping.get(mes, mes)

def registrar_carpeta_procesada(carpeta, num_archivos_procesados):
    with open(registro_path, 'a') as file:
        file.write(f"{carpeta}:{num_archivos_procesados}\n")

def carpeta_procesada(carpeta):
    if not os.path.exists(registro_path):
        return False
    with open(registro_path, 'r') as file:
        for line in file:
            if line.startswith(carpeta):
                return True
    return False

def obtener_fecha(df, engine):
    id_ns = df['id_ns'].iloc[0]
    year = df['year'].iloc[0]
    query = f"SELECT date FROM dates_growth_rates WHERE id_ns = '{id_ns}' AND year = '{year}';"
    fecha = pd.read_sql(query, engine)
    return fecha.iloc[0, 0] if not fecha.empty else None

def procesar_pdf(pdf_path):
    tables_dict_1 = {}  # Diccionario local para cada PDF
    table_counter = 1
    keyword_count = 0 

    filename = os.path.basename(pdf_path)
    id_ns_year_matches = re.findall(r'ns-(\d+)-(\d{4})', filename)
    if id_ns_year_matches:
        id_ns, year = id_ns_year_matches[0]
    else:
        print("No se encontraron coincidencias para id_ns y year en el nombre del archivo:", filename)
        return None, None, None, None, None  # Return None for tables_dict_1 as well

    new_filename = os.path.splitext(os.path.basename(pdf_path))[0].replace('-', '_')

    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages, 1):
            text = page.extract_text()
            if all(keyword in text for keyword in keywords):
                keyword_count += 1
                if keyword_count == 1:  # Solo procesar la primera ocurrencia
                    tables = tabula.read_pdf(pdf_path, pages=i, multiple_tables=False, stream=True) # change stream to another option if desired
                    for j, table_df in enumerate(tables, start=1):
                        nombre_dataframe = f"{new_filename}_{keyword_count}"
                        tables_dict_1[nombre_dataframe] = table_df
                        table_counter += 1

                    break  # Salir del bucle después de encontrar la primera ocurrencia

    return id_ns, year, tables_dict_1, keyword_count  # No retornar date aquí

def procesar_carpeta(carpeta, engine):
    print(f"Procesando la carpeta {os.path.basename(carpeta)}")
    pdf_files = [os.path.join(carpeta, f) for f in os.listdir(carpeta) if f.endswith('.pdf')]

    num_pdfs_procesados = 0
    num_dataframes_generados = 0

    table_counter = 1  # Inicializar el contador de tabla aquí
    tables_dict_1 = {}  # Declarar tables_dict_1 fuera del bucle principal
    
    for pdf_file in pdf_files:
        id_ns, year, tables_dict_temp, keyword_count = procesar_pdf(pdf_file)

        if tables_dict_temp:
            for nombre_df, df in tables_dict_temp.items():
                nombre_archivo = os.path.splitext(os.path.basename(pdf_file))[0].replace('-', '_')
                nombre_df = f"{nombre_archivo}_{keyword_count}"
                
                # Almacenar DataFrame sin procesar en tables_dict_1
                tables_dict_1[nombre_df] = df.copy()
                
                # Aplicar las 20 líneas de funciones de limpieza a una copia del DataFrame
                df_clean = df.copy()

                if any(col.isdigit() and len(col) == 4 for col in df_clean.columns):
                    # Si hay al menos una columna que representa un año
                    df_clean = swap_nan_se(df_clean)
                    df_clean = split_column_by_pattern(df_clean)
                    df_clean = drop_rare_caracter_row(df_clean)
                    df_clean = drop_nan_rows(df_clean)
                    df_clean = drop_nan_columns(df_clean)
                    df_clean = relocate_last_columns(df_clean)
                    df_clean = replace_first_dot(df_clean)
                    df_clean = swap_first_second_row(df_clean)
                    df_clean = drop_nan_rows(df_clean)
                    df_clean = reset_index(df_clean)
                    df_clean = remove_digit_slash(df_clean)
                    df_clean = replace_var_perc_first_column(df_clean)
                    df_clean = replace_var_perc_last_columns(df_clean)
                    df_clean = replace_number_moving_average(df_clean)
                    df_clean = separate_text_digits(df_clean)
                    df_clean = intercambiar_valores(df_clean)
                    df_clean = relocate_last_column(df_clean)
                    df_clean = clean_first_row(df_clean)
                    df_clean = find_year_column(df_clean)
                    year_columns = extract_years(df_clean)
                    df_clean = get_months_sublist_list(df_clean, year_columns)
                    df_clean = first_row_columns(df_clean)
                    df_clean = clean_columns_values(df_clean)
                    df_clean = convertir_float(df_clean)
                    df_clean = replace_set_sep(df_clean)
                    df_clean = spaces_se_es(df_clean)
                    df_clean = replace_services(df_clean)
                    df_clean = redondear_valores(df_clean, decimales=1)
                else: # 2014 ns 08
                    # Si no hay columnas que representen años
                    df_clean = revisar_primera_fila(df_clean)
                    df_clean = revisar_primera_fila_1(df_clean)
                    df_clean = replace_first_row_with_columns(df_clean)
                    df_clean = swap_nan_se(df_clean)
                    df_clean = split_column_by_pattern(df_clean)
                    df_clean = drop_rare_caracter_row(df_clean)
                    df_clean = drop_nan_rows(df_clean)
                    df_clean = drop_nan_columns(df_clean)
                    df_clean = relocate_last_columns(df_clean)
                    #df_clean = replace_first_dot(df_clean) # comment for 2014 ns 08
                    df_clean = swap_first_second_row(df_clean)
                    df_clean = drop_nan_rows(df_clean)
                    df_clean = reset_index(df_clean)
                    df_clean = remove_digit_slash(df_clean)
                    df_clean = replace_var_perc_first_column(df_clean)
                    df_clean = replace_var_perc_last_columns(df_clean)
                    df_clean = replace_number_moving_average(df_clean)
                    df_clean = expand_column(df_clean) # 2014 ns 08
                    df_clean = split_values_1(df_clean) # 2014 ns 08
                    df_clean = split_values_2(df_clean) # 2016 ns 15
                    df_clean = split_values_3(df_clean) # 2016 ns 19
                    df_clean = separate_text_digits(df_clean)
                    df_clean = intercambiar_valores(df_clean)
                    df_clean = relocate_last_column(df_clean)
                    df_clean = clean_first_row(df_clean)
                    df_clean = find_year_column(df_clean)
                    year_columns = extract_years(df_clean)
                    df_clean = get_months_sublist_list(df_clean, year_columns)
                    df_clean = first_row_columns(df_clean)
                    df_clean = clean_columns_values(df_clean)
                    df_clean = convertir_float(df_clean)
                    df_clean = replace_nan_with_previous_column_1(df_clean)
                    df_clean = replace_nan_with_previous_column_2(df_clean)
                    df_clean = replace_nan_with_previous_column_3(df_clean)
                    df_clean = replace_set_sep(df_clean)
                    df_clean = spaces_se_es(df_clean)
                    df_clean = replace_services(df_clean)
                    df_clean = redondear_valores(df_clean, decimales=1)
                
                # Añadir la columna 'year' al DataFrame limpio
                df_clean.insert(0, 'year', year)
                
                # Añadir la columna 'id_ns' al DataFrame limpio
                df_clean.insert(1, 'id_ns', id_ns)
                
                # Obtener la fecha correspondiente de la base de datos
                fecha = obtener_fecha(df_clean, engine)
                if fecha:
                    # Añadir la columna 'date' al DataFrame limpio
                    df_clean.insert(2, 'date', fecha)
                else:
                    print("No se encontró fecha en la base de datos para id_ns:", id_ns, "y year:", year)
                
                # Almacenar DataFrame limpio en dataframes_dict_1
                dataframes_dict_1[nombre_df] = df_clean

                print(f'  {table_counter}. El dataframe generado para el archivo {pdf_file} es: {nombre_df}')
                num_dataframes_generados += 1
                table_counter += 1  # Incrementar el contador de tabla aquí
        
        num_pdfs_procesados += 1  # Incrementar el número de PDFs procesados por cada PDF en la carpeta

    return num_pdfs_procesados, num_dataframes_generados, tables_dict_1


def procesar_carpetas():
    pdf_folder = 'input_pdf'
    carpetas = [os.path.join(pdf_folder, d) for d in os.listdir(pdf_folder) if os.path.isdir(os.path.join(pdf_folder, d))]
    
    tables_dict_1 = {}  # Inicializar tables_dict_1 aquí
    
    for carpeta in carpetas:
        if carpeta_procesada(carpeta):
            print(f"La carpeta {carpeta} ya ha sido procesada.")
            continue
        
        num_pdfs_procesados, num_dataframes_generados, tables_dict_temp = procesar_carpeta(carpeta, engine)
        
        # Actualizar tables_dict_1 con los valores devueltos de procesar_carpeta()
        tables_dict_1.update(tables_dict_temp)
        
        registrar_carpeta_procesada(carpeta, num_pdfs_procesados)

        # Preguntar al usuario si desea continuar con la siguiente carpeta
        root = Tk()
        root.withdraw()
        root.attributes('-topmost', True)  # Para asegurar que la ventana esté en primer plano
        
        mensaje = f"Se han generado {num_dataframes_generados} dataframes en la carpeta {carpeta}. ¿Deseas continuar con la siguiente carpeta?"
        continuar = messagebox.askyesno("Continuar", mensaje)
        root.destroy()

        if not continuar:
            print("Procesamiento detenido por el usuario.")
            break  # Romper el bucle for si el usuario decide no continuar

    print("Procesamiento completado para todas las carpetas.")  # Add a message to indicate completion

    return tables_dict_1  # Devolver tables_dict_1 al final de la función

if __name__ == "__main__":
    # Get environment variables
    user = os.environ.get('CIUP_SQL_USER')
    password = os.environ.get('CIUP_SQL_PASS')
    host = os.environ.get('CIUP_SQL_HOST')
    port = 5432
    database = 'gdp_revisions_datasets'

    # Check if all environment variables are defined
    if not all([host, user, password]):
        raise ValueError("Some environment variables are missing (CIUP_SQL_HOST, CIUP_SQL_USER, CIUP_SQL_PASS)")

    # Create connection string
    connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"

    # Create SQLAlchemy engine
    engine = create_engine(connection_string)

    tables_dict_1 = procesar_carpetas()  # Capturar el valor devuelto de procesar_carpetas()


In [ ]:
tables_dict_1.keys()

In [ ]:
dataframes_dict_1.keys()

In [ ]:
tables_dict_1['ns_01_2024_1'].head(5)

In [ ]:
df_1 = dataframes_dict_1['ns_01_2024_1']
df_1

In [ ]:
df_1[(df_1['sectores_economicos'] == 'agropecuario') | (df_1['economic_sectors'] == 'agriculture and livestock')]

# Table 2

# Con registro de carpetas procesdas 2

In [ ]:
import tabula
import pdfplumber
import pandas as pd
import numpy as np
import roman
import os
import re
from datetime import datetime
import locale
from tkinter import Tk, messagebox, TOP, YES, NO
import os


# Establecer la localización en español
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

# Palabras clave para buscar en el texto de la página
keywords = ["ECONOMIC SECTORS"]

# Diccionario para almacenar los DataFrames generados
dataframes_dict_2 = {}

# Ruta del archivo de registro de carpetas procesadas
registro_path = 'dataframes_record/carpetas_procesadas_2.txt'

# Función para corregir los nombres de los meses
def corregir_nombre_mes(mes):
    meses_mapping = {
        'setiembre': 'septiembre',
        # Agrega más mapeos si es necesario para otros nombres de meses
    }
    return meses_mapping.get(mes, mes)

def registrar_carpeta_procesada(carpeta, num_archivos_procesados):
    with open(registro_path, 'a') as file:
        file.write(f"{carpeta}:{num_archivos_procesados}\n")

def carpeta_procesada(carpeta):
    if not os.path.exists(registro_path):
        return False
    with open(registro_path, 'r') as file:
        for line in file:
            if line.startswith(carpeta):
                return True
    return False

def obtener_fecha(df, engine):
    id_ns = df['id_ns'].iloc[0]
    year = df['year'].iloc[0]
    query = f"SELECT date FROM dates_growth_rates WHERE id_ns = '{id_ns}' AND year = '{year}';"
    fecha = pd.read_sql(query, engine)
    return fecha.iloc[0, 0] if not fecha.empty else None

def procesar_pdf(pdf_path):
    tables_dict_2 = {}  # Diccionario local para cada PDF
    table_counter = 1
    keyword_count = 0 

    filename = os.path.basename(pdf_path)
    id_ns_year_matches = re.findall(r'ns-(\d+)-(\d{4})', filename)
    if id_ns_year_matches:
        id_ns, year = id_ns_year_matches[0]
    else:
        print("No se encontraron coincidencias para id_ns y year en el nombre del archivo:", filename)
        return None, None, None, None

    new_filename = os.path.splitext(os.path.basename(pdf_path))[0].replace('-', '_')

    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages, 1):
            text = page.extract_text()
            if all(keyword in text for keyword in keywords):
                keyword_count += 1
                if keyword_count == 2:
                    tables = tabula.read_pdf(pdf_path, pages=i, multiple_tables=False)
                    for j, table_df in enumerate(tables, start=1):
                        nombre_dataframe = f"{new_filename}_{keyword_count}"
                        tables_dict_2[nombre_dataframe] = table_df
                        table_counter += 1

    return id_ns, year, tables_dict_2, keyword_count


def procesar_carpeta(carpeta, engine):
    print(f"Procesando la carpeta {os.path.basename(carpeta)}")
    pdf_files = [os.path.join(carpeta, f) for f in os.listdir(carpeta) if f.endswith('.pdf')]

    num_pdfs_procesados = 0
    num_dataframes_generados = 0

    table_counter = 1  # Inicializar el contador de tabla aquí
    tables_dict_2 = {}  # Declarar tables_dict fuera del bucle principal
    
    for pdf_file in pdf_files:
        id_ns, year, tables_dict_temp, keyword_count = procesar_pdf(pdf_file)

        if tables_dict_temp:
            for nombre_df, df in tables_dict_temp.items():
                nombre_archivo = os.path.splitext(os.path.basename(pdf_file))[0].replace('-', '_')
                nombre_df = f"{nombre_archivo}_{keyword_count}"

                # Almacenar DataFrame sin procesar en tables_dict
                tables_dict_2[nombre_df] = df.copy()

                # Aplicar las 20 líneas de funciones de limpieza a una copia del DataFrame
                df_clean = df.copy()
                if df_clean.iloc[0, 0] is np.nan:
                    # Aplicar las 20 líneas de limpieza
                    df_clean = drop_nan_columns(df_clean)
                    df_clean = separate_years(df_clean)
                    df_clean = relocate_roman_numerals(df_clean)
                    df_clean = extract_mixed_values(df_clean)
                    df_clean = replace_first_row_nan(df_clean)
                    df_clean = first_row_columns(df_clean)
                    df_clean = swap_first_second_row(df_clean)
                    df_clean = reset_index(df_clean)
                    df_clean = drop_nan_row(df_clean)
                    year_columns = extract_years(df_clean)
                    df_clean = split_values(df_clean)
                    df_clean = separate_text_digits(df_clean)
                    df_clean = roman_arabic(df_clean)
                    df_clean = fix_duplicates(df_clean)
                    df_clean = relocate_last_column(df_clean)
                    df_clean = clean_first_row(df_clean)
                    df_clean = get_quarters_sublist_list(df_clean, year_columns)
                    df_clean = first_row_columns(df_clean)
                    df_clean = clean_columns_values(df_clean)
                    df_clean = reset_index(df_clean)
                    df_clean = convertir_float(df_clean)
                    df_clean = replace_set_sep(df_clean)
                    df_clean = spaces_se_es(df_clean)
                    df_clean = replace_services(df_clean)
                    df_clean = redondear_valores(df_clean, decimales=1)
                else:
                    # Aplicar las 15 líneas de limpieza
                    df_clean = exchange_roman_nan(df_clean)
                    df_clean = intercambiar_columnas(df_clean)
                    df_clean = drop_nan_columns(df_clean)
                    df_clean = remove_digit_slash(df_clean)
                    df_clean = last_column_es(df_clean)
                    df_clean = swap_first_second_row(df_clean)
                    df_clean = drop_nan_rows(df_clean)
                    df_clean = reset_index(df_clean)
                    year_columns = extract_years(df_clean)
                    df_clean = separate_text_digits(df_clean)
                    df_clean = roman_arabic(df_clean)
                    df_clean = fix_duplicates(df_clean)
                    df_clean = relocate_last_column(df_clean)
                    df_clean = clean_first_row(df_clean)
                    df_clean = get_quarters_sublist_list(df_clean, year_columns)
                    df_clean = first_row_columns(df_clean)
                    df_clean = clean_columns_values(df_clean)
                    df_clean = reset_index(df_clean)
                    df_clean = convertir_float(df_clean)
                    df_clean = replace_set_sep(df_clean)
                    df_clean = spaces_se_es(df_clean)
                    df_clean = replace_services(df_clean)
                    df_clean = redondear_valores(df_clean, decimales=1)

                # Añadir la columna 'year' al DataFrame limpio
                df_clean.insert(0, 'year', year)
                
                # Añadir la columna 'id_ns' al DataFrame limpio
                df_clean.insert(1, 'id_ns', id_ns)
                
                # Obtener la fecha correspondiente de la base de datos
                fecha = obtener_fecha(df_clean, engine)
                if fecha:
                    # Añadir la columna 'date' al DataFrame limpio
                    df_clean.insert(2, 'date', fecha)
                else:
                    print("No se encontró fecha en la base de datos para id_ns:", id_ns, "y year:", year)

                # Almacenar DataFrame limpio en dataframes_dict
                dataframes_dict_2[nombre_df] = df_clean

                print(f'  {table_counter}. El dataframe generado para el archivo {pdf_file} es: {nombre_df}')
                num_dataframes_generados += 1
                table_counter += 1  # Incrementar el contador de tabla aquí
                    
        num_pdfs_procesados += 1  # Incrementar el número de PDFs procesados por cada PDF en la carpeta

    return num_pdfs_procesados, num_dataframes_generados, tables_dict_2
        
def procesar_carpetas():
    pdf_folder = 'input_pdf'
    carpetas = [os.path.join(pdf_folder, d) for d in os.listdir(pdf_folder) if os.path.isdir(os.path.join(pdf_folder, d))]

    tables_dict_2 = {}  # Inicializar tables_dict aquí
    
    for carpeta in carpetas:
        if carpeta_procesada(carpeta):
            print(f"La carpeta {carpeta} ya ha sido procesada.")
            continue
        
        num_pdfs_procesados, num_dataframes_generados, tables_dict_temp = procesar_carpeta(carpeta, engine)
        
        # Actualizar tables_dict con los valores devueltos de procesar_carpeta()
        tables_dict_2.update(tables_dict_temp)
        
        registrar_carpeta_procesada(carpeta, num_pdfs_procesados)

        # Preguntar al usuario si desea continuar con la siguiente carpeta
        root = Tk()
        root.withdraw()
        root.attributes('-topmost', True)  # Para asegurar que la ventana esté en primer plano
        
        mensaje = f"Se han generado {num_dataframes_generados} dataframes en la carpeta {carpeta}. ¿Deseas continuar con la siguiente carpeta?"
        continuar = messagebox.askyesno("Continuar", mensaje)
        root.destroy()

        if not continuar:
            print("Procesamiento detenido por el usuario.")
            break  # Romper el bucle for si el usuario decide no continuar

    print("Procesamiento completado para todas las carpetas.")  # Add a message to indicate completion

    return tables_dict_2  # Devolver tables_dict al final de la función
    
if __name__ == "__main__":
    # Get environment variables
    user = os.environ.get('CIUP_SQL_USER')
    password = os.environ.get('CIUP_SQL_PASS')
    host = os.environ.get('CIUP_SQL_HOST')
    port = 5432
    database = 'gdp_revisions_datasets'

    # Check if all environment variables are defined
    if not all([host, user, password]):
        raise ValueError("Some environment variables are missing (CIUP_SQL_HOST, CIUP_SQL_USER, CIUP_SQL_PASS)")

    # Create connection string
    connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"

    # Create SQLAlchemy engine
    engine = create_engine(connection_string)
    tables_dict_2 = procesar_carpetas() # Capturar el valor devuelto de procesar_carpetas()

In [ ]:
tables_dict_2.keys()

In [ ]:
dataframes_dict_2.keys()

In [ ]:
tables_dict_2['ns_01_2024_2'].head(5)

In [ ]:
dataframes_dict_2['ns_01_2024_2']

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px">
    Back to the
    <a href="#outilne" style="color: #3d30a2;">
    outline.
    </a>
    <div/>

<div id="3">
   <!-- Contenido de la celda de destino -->
</div>

<h1><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">3.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">SQL Tables</span></h1>

<div style="font-family: charter; text-align: left; color:dark">
    Finally, after obtaining and cleaning all the necessary data, we can create the three most important datasets to store realeses, vintages, and revisions. These datasets will be stored as tables in SQL and can be loaded into any software or programming language.
    <div/>

<div id="sector">
   <!-- Contenido de la celda de destino -->
</div>

# Chose sector_economico and economic_sector

In [ ]:
import tkinter as tk

# Definir la lista de opciones
opciones = [
    "pbi",
    "agropecuario",
    "pesca",
    "mineria e hidrocarburos",
    "manufactura",
    "electricidad y agua",
    "construccion",
    "comercio",
    "otros servicios"
]

# Función para guardar la opción seleccionada y cerrar la ventana
def guardar_opcion():
    global sector_economico
    sector_economico = opcion_seleccionada.get()
    root.destroy()  # Cerrar la ventana después de seleccionar una opción

# Crear la ventana emergente
root = tk.Tk()
root.title("Seleccionar opción")

# Variable para almacenar la opción seleccionada
opcion_seleccionada = tk.StringVar(root)
opcion_seleccionada.set(opciones[0])  # Opción predeterminada

# Crear el menú de opciones
menu = tk.OptionMenu(root, opcion_seleccionada, *opciones)
menu.pack(pady=10)

# Botón para confirmar la selección
boton_confirmar = tk.Button(root, text="Confirmar", command=guardar_opcion)
boton_confirmar.pack()

# Mostrar la ventana
root.update_idletasks()
root.wait_window()

# Mostrar el valor seleccionado
print("Sector económico seleccionado:", opcion_seleccionada.get())


In [ ]:
import tkinter as tk

# Definir la lista de opciones
opciones = [
    "gdp",
    "agriculture and livestock",
    "fishing",
    "mining and fuel",
    "manufacturing",
    "electricity and water",
    "construction",
    "commerce",
    "other services"
]

# Función para guardar la opción seleccionada y cerrar la ventana
def guardar_opcion():
    global economic_sector
    economic_sector = opcion_seleccionada.get()
    root.destroy()  # Cerrar la ventana después de seleccionar una opción

# Crear la ventana emergente
root = tk.Tk()
root.title("Seleccionar opción")

# Variable para almacenar la opción seleccionada
opcion_seleccionada = tk.StringVar(root)
opcion_seleccionada.set(opciones[0])  # Opción predeterminada

# Crear el menú de opciones
menu = tk.OptionMenu(root, opcion_seleccionada, *opciones)
menu.pack(pady=10)

# Botón para confirmar la selección
boton_confirmar = tk.Button(root, text="Confirmar", command=guardar_opcion)
boton_confirmar.pack()

# Mostrar la ventana
root.update_idletasks()
root.wait_window()

# Mostrar el valor seleccionado
print("Sector económico seleccionado:", opcion_seleccionada.get())

# Chose the year and label datatset

In [ ]:
import tkinter as tk
from tkinter import simpledialog

# Crear una ventana principal
root = tk.Tk()
root.withdraw()  # Ocultar la ventana principal

# Pedir al usuario que introduzca el valor de sector_economico
sector = simpledialog.askstring("Sector Económico", "Introduce el valor del sector:")

# Pedir al usuario que introduzca el valor de economic_sector
#year = simpledialog.askstring("Year", "Introduce el valor de year:")

# Mostrar los valores introducidos por el usuario
print("Valor del sector:", sector)
#print("Valor de economic_sector:", year)


<div id="3-1">
   <!-- Contenido de la celda de destino -->
</div>

<h2><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">3.1.</span>
    <span style = "color: dark; font-family: PT Serif Pro Book;">
    Annual Concatenation
    </span>
    </h2>

In [ ]:
def concatenate_annual_df(dataframes_dict):
    # List to store the names of dataframes that meet the criterion of ending in '_2'
    dataframes_ending_with_2 = []

    # List to store the names of dataframes to be concatenated
    dataframes_to_concatenate = []

    # Iterate over the dataframe names in the all_dataframes dictionary
    for df_name in dataframes_dict.keys():
        # Check if the dataframe name ends with '_2' and add it to the corresponding list
        if df_name.endswith('_2'):
            dataframes_ending_with_2.append(df_name)
            dataframes_to_concatenate.append(dataframes_dict[df_name])

    # Print the names of dataframes that meet the criterion of ending in '_2'
    print("DataFrames ending with '_2' that will be concatenated:")
    for df_name in dataframes_ending_with_2:
        print(df_name)

    # Concatenate all dataframes in the 'dataframes_to_concatenate' list
    if dataframes_to_concatenate:
        # Concatenate only rows that meet the specified conditions
        annual_growth_rates = pd.concat([df[(df['sectores_economicos'] == sector_economico) | (df['economic_sectors'] == economic_sector)] 
                                    for df in dataframes_to_concatenate 
                                    if 'sectores_economicos' in df.columns and 'economic_sectors' in df.columns], 
                                    ignore_index=True)

        # Keep only columns that start with 'year' and the 'id_ns', 'year', and 'date' columns
        columns_to_keep = ['year', 'id_ns', 'date'] + [col for col in annual_growth_rates.columns if col.endswith('_year')]

        # Drop unwanted columns
        annual_growth_rates = annual_growth_rates[columns_to_keep]
        
        # Remove duplicate columns if any
        annual_growth_rates = annual_growth_rates.loc[:,~annual_growth_rates.columns.duplicated()]
    
        # Cambia el nombre de las columnas a partir de la cuarta columna
        annual_growth_rates.columns = [col.split('_')[1] + '_' + col.split('_')[0] if '_' in col and idx >= 3 else col for idx, col in enumerate(annual_growth_rates.columns)]

        # Print the number of rows in the concatenated dataframe
        print("Number of rows in the concatenated dataframe:", len(annual_growth_rates))
        
        return annual_growth_rates
    else:
        print("No dataframes were found to concatenate.")
        return None

In [ ]:
globals()[f"{sector}_annual_growth_rates"] = concatenate_annual_df(dataframes_dict_2)

In [ ]:
pd.set_option('display.max_rows', None)
globals()[f"{sector}_annual_growth_rates"]

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px">
    Back to the
    <a href="#outilne" style="color: #3d30a2;">
    outline.
    </a>
    <div/>

<div id="3-2">
   <!-- Contenido de la celda de destino -->
</div>

<h2><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">3.2.</span>
    <span style = "color: dark; font-family: PT Serif Pro Book;">
    Quarterly Concatenation
    </span>
    </h2>

In [ ]:
import pandas as pd

def concatenate_quarterly_df(dataframes_dict):
    # List to store the names of dataframes that meet the criterion of ending in '_2'
    dataframes_ending_with_2 = []

    # List to store the names of dataframes to be concatenated
    dataframes_to_concatenate = []

    # Iterate over the dataframe names in the all_dataframes dictionary
    for df_name in dataframes_dict.keys():
        # Check if the dataframe name ends with '_2' and add it to the corresponding list
        if df_name.endswith('_2'):
            dataframes_ending_with_2.append(df_name)
            dataframes_to_concatenate.append(dataframes_dict[df_name])

    # Print the names of dataframes that meet the criterion of ending in '_2'
    print("DataFrames ending with '_2' that will be concatenated:")
    for df_name in dataframes_ending_with_2:
        print(df_name)

    # Concatenate all dataframes in the 'dataframes_to_concatenate' list
    if dataframes_to_concatenate:
        # Concatenate only rows that meet the specified conditions
        quarterly_growth_rates = pd.concat([df[(df['sectores_economicos'] == sector_economico) | (df['economic_sectors'] == economic_sector)] 
                                    for df in dataframes_to_concatenate 
                                    if 'sectores_economicos' in df.columns and 'economic_sectors' in df.columns], 
                                    ignore_index=True)

        # Keep all columns except those starting with 'year_', in addition to the 'id_ns', 'year', and 'date' columns
        columns_to_keep = ['year', 'id_ns', 'date'] + [col for col in quarterly_growth_rates.columns if not col.endswith('_year')]

        # Select unwanted columns
        quarterly_growth_rates = quarterly_growth_rates[columns_to_keep]

        # Drop the 'sectores_economicos' and 'economic_sectors' columns
        quarterly_growth_rates.drop(columns=['sectores_economicos', 'economic_sectors'], inplace=True)

        # Remove duplicate columns if any
        quarterly_growth_rates = quarterly_growth_rates.loc[:,~quarterly_growth_rates.columns.duplicated()]

        # Cambia el nombre de las columnas a partir de la cuarta columna
        #quarterly_growth_rates.columns = [col.split('_')[0] + '_q' + col.split('_')[1] if '_' in col and idx >= 3 else col
        #for idx, col in enumerate(quarterly_growth_rates.columns)]
        
        # Print the number of rows in the concatenated dataframe
        print("Number of rows in the concatenated dataframe:", len(quarterly_growth_rates))
        
        return quarterly_growth_rates
    else:
        print("No dataframes were found to concatenate.")
        return None

In [ ]:
globals()[f"{sector}_quarterly_growth_rates"] = concatenate_quarterly_df(dataframes_dict_2)

In [ ]:
pd.set_option('display.max_rows', None)
globals()[f"{sector}_quarterly_growth_rates"]

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px">
    Back to the
    <a href="#outilne" style="color: #3d30a2;">
    outline.
    </a>
    <div/>

<div id="3-3">
   <!-- Contenido de la celda de destino -->
</div>

<h2><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">3.3.</span>
    <span style = "color: dark; font-family: PT Serif Pro Book;">
    Monthly Concatenation
    </span>
    </h2>

In [ ]:
import pandas as pd

def concatenate_monthly_df(dataframes_dict):
    # List to store the names of dataframes that meet the criterion of ending in '_1'
    dataframes_ending_with_1 = []

    # List to store the names of dataframes to be concatenated
    dataframes_to_concatenate = []

    # Iterate over the dataframe names in the all_dataframes dictionary
    for df_name in dataframes_dict.keys():
        # Check if the dataframe name ends with '_1' and add it to the corresponding list
        if df_name.endswith('_1'):
            dataframes_ending_with_1.append(df_name)
            dataframes_to_concatenate.append(dataframes_dict[df_name])

    # Print the names of dataframes that meet the criterion of ending with '_1'
    print("DataFrames ending with '_1' that will be concatenated:")
    for df_name in dataframes_ending_with_1:
        print(df_name)

    # Concatenate all dataframes in the 'dataframes_to_concatenate' list
    if dataframes_to_concatenate:
        # Concatenate only rows that meet the specified conditions
        monthly_growth_rates = pd.concat([df[(df['sectores_economicos'] == sector_economico) | (df['economic_sectors'] == economic_sector)] 
                                    for df in dataframes_to_concatenate 
                                    if 'sectores_economicos' in df.columns and 'economic_sectors' in df.columns], 
                                    ignore_index=True)

        # Keep all columns except those starting with 'year_', in addition to the 'id_ns', 'year', and 'date' columns
        columns_to_keep = ['year', 'id_ns', 'date'] + [col for col in monthly_growth_rates.columns if not col.endswith('_year')]

        # Select unwanted columns
        monthly_growth_rates = monthly_growth_rates[columns_to_keep]

        # Drop the 'sectores_economicos' and 'economic_sectors' columns
        monthly_growth_rates.drop(columns=['sectores_economicos', 'economic_sectors'], inplace=True)

        # Remove duplicate columns if any
        monthly_growth_rates = monthly_growth_rates.loc[:,~monthly_growth_rates.columns.duplicated()]
        
        # Drop columns with at least two underscores in their names
        columns_to_drop = [col for col in monthly_growth_rates.columns if col.count('_') >= 2]
        monthly_growth_rates.drop(columns=columns_to_drop, inplace=True)
        
        # Cambia el nombre de las columnas a partir de la cuarta columna
        monthly_growth_rates.columns = [col.split('_')[1] + '_' + col.split('_')[0] if '_' in col and idx >= 3 else col for idx, col in enumerate(monthly_growth_rates.columns)]
        
        # Diccionario de mapeo de nombres de meses
        #meses = {
        #    'ene': 'm1', 'feb': 'm2', 'mar': 'm3', 'abr': 'm4',
        #    'may': 'm5', 'jun': 'm6', 'jul': 'm7', 'ago': 'm8',
        #    'sep': 'm9', 'oct': 'm10', 'nov': 'm11', 'dic': 'm12'
        #}
        
        # Función para reemplazar las claves por los valores del diccionario en el nombre de las columnas
        #def replace_months(column_name, meses):
        #    for key, value in meses.items():
        #        if key in column_name:
        #            return column_name.replace(key, value)
        #    return column_name

        # Aplicar la función a todas las columnas del DataFrame
        #monthly_growth_rates.columns = monthly_growth_rates.columns.map(lambda x: replace_months(x, meses))

        # Print the number of rows in the concatenated dataframe
        print("Number of rows in the concatenated dataframe:", len(monthly_growth_rates))

        return monthly_growth_rates


In [ ]:
globals()[f"{sector}_monthly_growth_rates"] = concatenate_monthly_df(dataframes_dict_1)

In [ ]:
pd.set_option('display.max_rows', None)
globals()[f"{sector}_monthly_growth_rates"]

<div style="background-color: #1E3B58; color: white; padding: 10px;">
    <h2>Create revision dataset (annual)</h2>
</div>

In [ ]:
sector

In [ ]:
# 1. Calcular la diferencia entre el último y el primer valor no NaN para cada columna, excepto 'year', 'ns_id' y 'date'
revision = globals()[f"{sector}_annual_growth_rates"].drop(columns=['year', 'id_ns', 'date']).apply(lambda x: x.loc[x.last_valid_index()] - x.loc[x.first_valid_index()])


#  Nuevo

In [ ]:
# 2. Crear un nuevo DataFrame con los resultados
globals()[f"{sector}_annual_revisions"] = pd.DataFrame({'revision_date': revision.index, f'{sector}_revision': revision.values})
# Suponiendo que 'gdp_monthly_growth_rates' es tu DataFrame
pd.set_option('display.max_rows', None)
globals()[f"{sector}_annual_revisions"]

# Para graficar en Python

In [ ]:
import pandas as pd

# Extraer el año de la cadena y convertirlo a un tipo entero
globals()[f"{sector}_annual_revisions"]['year'] = globals()[f"{sector}_annual_revisions"]['revision_date'].str.extract(r'(\d+)')
globals()[f"{sector}_annual_revisions"]['year'] = globals()[f"{sector}_annual_revisions"]['year'].astype(int)

# Crear una nueva columna de tipo fecha
globals()[f"{sector}_annual_revisions"]['revision_date'] = pd.to_datetime(globals()[f"{sector}_annual_revisions"]['year'], format='%Y')

# Eliminar la columna 'year' si ya no es necesaria
globals()[f"{sector}_annual_revisions"].drop(columns=['year'], inplace=True)

# Mostrar el resultado
globals()[f"{sector}_annual_revisions"]


<div style="background-color: #1E3B58; color: white; padding: 10px;">
    <h2>Create revision dataset (quarterly)</h2>
</div>

In [ ]:
sector

In [ ]:
# 1. Calcular la diferencia entre el último y el primer valor no NaN para cada columna, excepto 'year', 'ns_id' y 'date'
revision = globals()[f"{sector}_quarterly_growth_rates"].drop(columns=['year', 'id_ns', 'date']).apply(lambda x: x.loc[x.last_valid_index()] - x.loc[x.first_valid_index()])


#  Nuevo

In [ ]:
# 2. Crear un nuevo DataFrame con los resultados
globals()[f"{sector}_quarterly_revisions"] = pd.DataFrame({'revision_date': revision.index, f'{sector}_revision': revision.values})
# Suponiendo que 'gdp_monthly_growth_rates' es tu DataFrame
pd.set_option('display.max_rows', None)
globals()[f"{sector}_quarterly_revisions"]

# Para graficar en Python

In [ ]:
import pandas as pd

# Convertir la columna 'revision_date' a tipo de datos de fecha
globals()[f"{sector}_quarterly_revisions"]['revision_date'] = pd.to_datetime(globals()[f"{sector}_quarterly_revisions"]['revision_date'], format='%Y_%m')

# Mostrar el resultado
globals()[f"{sector}_quarterly_revisions"]

<div style="background-color: #1E3B58; color: white; padding: 10px;">
    <h2>Create revision dataset (monthly)</h2>
</div>

In [ ]:
sector

In [ ]:
# 1. Calcular la diferencia entre el último y el primer valor no NaN para cada columna, excepto 'year', 'ns_id' y 'date'
revision = globals()[f"{sector}_monthly_growth_rates"].drop(columns=['year', 'id_ns', 'date']).apply(lambda x: x.loc[x.last_valid_index()] - x.loc[x.first_valid_index()])


#  Nuevo

In [ ]:
# 2. Crear un nuevo DataFrame con los resultados
globals()[f"{sector}_monthly_revisions"] = pd.DataFrame({'revision_date': revision.index, f'{sector}_revision': revision.values})
# Suponiendo que 'gdp_monthly_growth_rates' es tu DataFrame
pd.set_option('display.max_rows', None)
globals()[f"{sector}_monthly_revisions"]

# Para graficar en python

In [ ]:
import pandas as pd

# Extraer el mes y el año de la columna 'revision_date'
globals()[f"{sector}_monthly_revisions"]['month'] = globals()[f"{sector}_monthly_revisions"]['revision_date'].str.split('_').str[0]
globals()[f"{sector}_monthly_revisions"]['year'] = globals()[f"{sector}_monthly_revisions"]['revision_date'].str.split('_').str[1]

# Mapear los nombres de los meses a sus respectivos números
month_mapping = {
    'ene': '01', 'feb': '02', 'mar': '03', 'abr': '04',
    'may': '05', 'jun': '06', 'jul': '07', 'ago': '08',
    'sep': '09', 'oct': '10', 'nov': '11', 'dic': '12'
}

globals()[f"{sector}_monthly_revisions"]['month'] = globals()[f"{sector}_monthly_revisions"]['month'].map(month_mapping)

# Crear una nueva columna con la fecha en formato YYYY-MM-DD
globals()[f"{sector}_monthly_revisions"]['revision_date'] = globals()[f"{sector}_monthly_revisions"]['year'] + '-' + globals()[f"{sector}_monthly_revisions"]['month']

# Convertir la columna 'revision_date' a tipo de datos de fecha
globals()[f"{sector}_monthly_revisions"]['revision_date'] = pd.to_datetime(globals()[f"{sector}_monthly_revisions"]['revision_date'], format='%Y-%m')

# Eliminar columnas temporales 'month' y 'year'
globals()[f"{sector}_monthly_revisions"].drop(['month', 'year'], axis=1, inplace=True)

# Mostrar el resultado
globals()[f"{sector}_monthly_revisions"]

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px">
    Back to the
    <a href="#outilne" style="color: #3d30a2;">
    outline.
    </a>
    <div/>

<div id="3-4">
   <!-- Contenido de la celda de destino -->
</div>

<h2><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">3.4.</span>
    <span style = "color: dark; font-family: PT Serif Pro Book;">
    Loading SQL
    </span>
    </h2>

# Growth Rates

In [ ]:
import os
from sqlalchemy import create_engine

# Get environment variables
user = os.environ.get('CIUP_SQL_USER')
password = os.environ.get('CIUP_SQL_PASS')
host = os.environ.get('CIUP_SQL_HOST')
port = 5432
database = 'gdp_revisions_datasets'

# Check if all environment variables are defined
if not all([host, user, password]):
    raise ValueError("Some environment variables are missing (CIUP_SQL_HOST, CIUP_SQL_USER, CIUP_SQL_PASS)")

# Create connection string
connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"

# Create SQLAlchemy engine
engine = create_engine(connection_string)

# gdp_monthly_growth_rates is the DataFrame you want to save to the database
#gdp_annual_growth_rates.to_sql('gdp_annual_growth_rates_2013', engine, index=False, if_exists='replace')
#gdp_quarterly_growth_rates.to_sql('gdp_quarterly_growth_rates', engine, index=False, if_exists='replace')
#gdp_monthly_growth_rates.to_sql(f'{sector}_monthly_growth_rates', engine, index=False, if_exists='replace')

# REVISIONES

globals()[f"{sector}_monthly_growth_rates"].to_sql(f'{sector}_monthly_growth_rates', engine, index=False, if_exists='replace')
#globals()[f"{sector}_quarterly_growth_rates"].to_sql(f'{sector}_quarterly_growth_rates', engine, index=False, if_exists='replace')
#globals()[f"{sector}_annual_growth_rates"].to_sql(f'{sector}_annual_growth_rates', engine, index=False, if_exists='replace')

# Revisions

In [ ]:
import os
from sqlalchemy import create_engine

# Get environment variables
user = os.environ.get('CIUP_SQL_USER')
password = os.environ.get('CIUP_SQL_PASS')
host = os.environ.get('CIUP_SQL_HOST')
port = 5432
database = 'gdp_revisions_datasets'

# Check if all environment variables are defined
if not all([host, user, password]):
    raise ValueError("Some environment variables are missing (CIUP_SQL_HOST, CIUP_SQL_USER, CIUP_SQL_PASS)")

# Create connection string
connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"

# Create SQLAlchemy engine
engine = create_engine(connection_string)

# gdp_monthly_growth_rates is the DataFrame you want to save to the database
#gdp_annual_growth_rates.to_sql('gdp_annual_growth_rates_2013', engine, index=False, if_exists='replace')
#gdp_quarterly_growth_rates.to_sql('gdp_quarterly_growth_rates', engine, index=False, if_exists='replace')
#gdp_monthly_growth_rates.to_sql(f'{sector}_monthly_growth_rates', engine, index=False, if_exists='replace')

# REVISIONES

globals()[f"{sector}_monthly_revisions"].to_sql(f'{sector}_monthly_revisions', engine, index=False, if_exists='replace')
#globals()[f"{sector}_quarterly_revisions"].to_sql(f'{sector}_quarterly_revisions', engine, index=False, if_exists='replace')
#globals()[f"{sector}_annual_revisions"].to_sql(f'{sector}_annual_revisions', engine, index=False, if_exists='replace')

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px">
    Back to the
    <a href="#outilne" style="color: #3d30a2;">
    outline.
    </a>
    <div/>

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px">
    Back to the
    <a href="#outilne" style="color: #3d30a2;">
    outline.
    </a>
    <div/>

---
---
---